# Experiment 4 - Scripted yet natural agent 
Objective: Develop an AI agent that adheres to a script but also maintains natural conversational tones.

## Setup, imports, logging, keys

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
import openai


In [ ]:
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key  = os.environ['OPENAI_API_KEY']

# Step 1 - Setup helper functions

This first function will be a completion given a set of messages like the following, which shows how you can send entire context to gpt which will allow it to respond very differently than if it did not have context. The following roles are available: user, assisstamt and system. 

    messages =  [  
    {'role':'system', 'content':'You are friendly chatbot.'},
    {'role':'user', 'content':'Hi, my name is Isa'},
    {'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
    Is there anything I can help you with today?"},
    {'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
    response = get_completion_from_messages(messages, temperature=1)
    print(response)

In [ ]:

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

The following function will continue to get context from the chat text field that will be setup further down:

In [ ]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)


# Step 2 - Show how the messages technique works

Below we see the multiple messages as context example in action



In [ ]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Miguel'},
{'role':'assistant', 'content': "Hi Miguel! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

# Step 3 - Mini Chat bot with POC system prompt

Here we will simulate a mini bot with a fake system prompt that guides the user through a fake process of claims submission. 

In [ ]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are OneHealthBot, a conversational bot that guides the customer through \
the health insurance claims submission process.  On the first interaction \
after you greet the customer you will describe what the process will be like \
that you are going to walk the customer though. Enlist step by step and make it \
very concise and brief but explain all the steps.

This is the process, and the information you need to gather from user. 

1. Get the date of the procedure
2. Get the  type of procedure
3. Name of the person in claim

Do not let the conversation diverge, you need to wait until you have all \
those 3 pieces of information. If the conversation starts diverging, answer the \
question lightly, but quickly guide the user back in a very polite way to the next step. 

When you do, summarize the information. Ask the user to confirm \
if they want to submit the claim.

Submit the claim, thank the customer and ask the customer what else they \
need help with

"""} ]  # accumulate messages

inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

# Step 4 - Extract JSON

The following example shows how we can extract the JSON at the very end of the conversation. One cna imagine how this can turn into functions. 

In [20]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of claim submission information.\
 The fields should be 1) procedure-date 2) claimant-name 3) procedure-type'},    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
print(response)

Certainly! Here is a JSON summary of the claim submission information:

{
  "procedure-date": "12/23/2022",
  "claimant-name": "Miguel Alvarado",
  "procedure-type": "Back surgery"
}

Please let me know if there's anything else I can help you with!
